In [1]:
%load_ext autoreload

In [57]:
%autoreload 2
import pandas as pd
from os import path
import glob
import warnings
from scrapers.scrapepool import ScrapePool
from scrapers.wos import WosScraper
import linecache
import csv
from beakerx import *

# disable warnings, because pandas throws warnings for mixed type columns all the time..
warnings.filterwarnings('ignore')

%autoreload

In [3]:
# scraper settings
args = {
    'phantomjs_exec': 'C:\\phantomjs.exe',
    'urls': {
        #'author_search': "http://apps.webofknowledge.com/WOS_AuthorSearch.do?product=WOS&action=Search&search_mode=AuthorSearch&AU=",
        'author_search': "http://apps.webofknowledge.com/DaisyOneClickSearchNoHistory.do?search_mode=DaisyOneClickSearchNoHistory&parentProduct=WOS&parentQid=10036&product=WOS&author_name=",
        'save_to': 'data\\wos\\out_feb18\\'
    },
    'topic': "standardization OR standardisation",
    'num_pubs': 5
}

In [6]:
df = pd.read_csv("data/wos/experts_knut.csv", sep=",")#, quotechar="\"", encoding="ISO-8859-1")
searched = set(["_".join(".".join(path.basename(f).split(".")[:-1]).split("_")[:-1]) for f in glob.glob(path.join(args['urls']['save_to'], "*.*")) if path.getsize(f) > 0])
keywords = list(set(df['name'].tolist())-searched)
print(str(len(keywords)))
#SID=F1zvccKnI6GLoDk33zb

0


In [5]:
s = ScrapePool(WosScraper, keywords, args, chunk_size=200)
results = s.run()

connection tested successfully!
found 4 addresses for LAMBERT C: Claude.lambert@uni-st-etienne.fr, claude.lambert@chu-st-etienne.fr, claude.lambert@uni-st-etienne.fr, jvitte@ap-hm.fr
found 4 addresses for LINDERHOLM HW: hansl@gvc.gu.se, Peng.Zhang@gvc.gu.se, peng.zhang@gvc.gu.se, yinh@cma.gov.cn
found 4 addresses for FEDRIGO M: elena.mattia80@gmail.com, annalisa.angelini@unipd.it, lauro.meneghel@studenti.unipd.it, gjbeny@stanford.edu
found 3 addresses for CARDOZO L: sushmasrikrishna@hotmail.com, zefxis@yahoo.co.uk, alexander.von.gontard@uks.eu
found 7 addresses for SEMPOS CT: pmb22@cornell.edu, DavisCI@nih.gov, SemposCH@nih.gov, stevebrooks@hc-sc.gc.ca, semposch@mail.nih.gov, TaylorCL3@od.nih.gov, b.carter1@which.net
found 3 addresses for GRIMWADE D: david.grimwade@genetics.kcl.ac.uk, anthony.bench@addenbrookes.nhs.uk, susanna.akiki@bwhct.nhs.uk
found 4 addresses for KANG HN: mei.ho@nibsc.org, knezevici@who.int, kangh@who.int, Meenu.Wadhwa@nibsc.org
found 3 addresses for BEYER T: thoma

In [7]:
merged = dict()
for f in glob.glob(path.join(args['urls']['save_to'], "*.*")):
    if path.getsize(f) > 4:
        name= "_".join(".".join(path.basename(f).split(".")[:-1]).split("_")[:-1])
        merged[name] = linecache.getline(f, 2).replace("\"","").replace(";",",").rstrip()

In [ ]:
df = df.set_index('name')
df

In [8]:
df2 = pd.DataFrame.from_dict(merged, orient='index')
df2.columns = ['email']
df2

,email
ABBASI S,"f.mokhtarian@surrey.ac.uk,F.Mokhtarian@surrey...."
ABOAL JR,"zulema.varela@usc.es,valeria.spagnuolo@unina.i..."
ABRAMS P,"c.r.chapple@sheffield.ac.uk,paul.abrams@bui.ac..."
ACHENBACH P,"hauck@helmholtz-muenchen.de,castell@helmholtz-..."
ADAM T,"dr-thomas-adam@gmx.net,adamt@who.int,jbrycedan..."
ADAMSON GD,"christian.becker@obs-gyn.ox.ac.uk,icmart@icsev..."
ADAN A,"rbou@hsjdbcn.org,rblanco@humv.es,miguelaggay@h..."
AEGERTER J,Hellmuth.klingelhoeffer@bam.de
AGNER T,"jungersted@gmail.com,Jungersted@gmail.com,Joha..."
AGUILERA-CARACUEL J,"jaragon99@gmail.com,javieraguilera@ugr.es,nhur..."


In [ ]:
all_merged = pd.concat([df, df2], axis=1, join_axes=[df.index])
all_merged

In [43]:
def get_main_email(x):
    all_mails = [(e, e.split("@")[0]) for e in x["email"].split(",")]
    return ",".join([e for e, en in all_mails if x["lastname"].lower() in en.lower()])

In [44]:
df2["name"] = df2.index
df2["lastname"] = df2.apply(lambda x: x["name"].split(" ")[0].lower(), axis=1)
df2["main_email"] = df2.apply(get_main_email, axis=1)
df2

,email,main_email,name,lastname
ABBASI S,"f.mokhtarian@surrey.ac.uk,F.Mokhtarian@surrey....",abbasis@uphs.upenn.edu,ABBASI S,abbasi
ABOAL JR,"zulema.varela@usc.es,valeria.spagnuolo@unina.i...",bfjaboal@usc.es,ABOAL JR,aboal
ABRAMS P,"c.r.chapple@sheffield.ac.uk,paul.abrams@bui.ac...",paul.abrams@bui.ac.uk,ABRAMS P,abrams
ACHENBACH P,"hauck@helmholtz-muenchen.de,castell@helmholtz-...",peter.achenbach@helmholtz-muenchen.de,ACHENBACH P,achenbach
ADAM T,"dr-thomas-adam@gmx.net,adamt@who.int,jbrycedan...","dr-thomas-adam@gmx.net,adamt@who.int",ADAM T,adam
ADAMSON GD,"christian.becker@obs-gyn.ox.ac.uk,icmart@icsev...",,ADAMSON GD,adamson
ADAN A,"rbou@hsjdbcn.org,rblanco@humv.es,miguelaggay@h...",,ADAN A,adan
AEGERTER J,Hellmuth.klingelhoeffer@bam.de,,AEGERTER J,aegerter
AGNER T,"jungersted@gmail.com,Jungersted@gmail.com,Joha...",,AGNER T,agner
AGUILERA-CARACUEL J,"jaragon99@gmail.com,javieraguilera@ugr.es,nhur...",,AGUILERA-CARACUEL J,aguilera-caracuel


In [45]:
df2.to_csv(path.join(args['urls']['save_to'], 'merged.csv'), sep=";", index=True, quotechar='"', quoting=csv.QUOTE_ALL)

## sent

In [64]:
df_sent = pd.read_csv("data/wos/sent.csv", sep=",")
df_main_auth = pd.DataFrame(["ydargaud@univ-lyon1.fr","c.caltagirone@hsantalucia.it","lianglbh@126.com","patrick.white@kcl.ac.uk","a.j.k.williams@bristol.ac.uk","karen.bieback@medma.uni-heidelberg.de","edwards.davidjohn@gmail.com","fpereirapt@gmail.com","jpalmeida@inf.ufes.br","dirk.roggenbuck@hs-lausitz.de","passalacqua@unige.it","wolfgang.uter@fau.de","karen.phinney@nist.gov","polly.bingley@bristol.ac.uk","joachim.fluhr@charite.de","m.y.berger@umcg.nl","lina.pezzuti@uniroma1.it","d.finlay@ulster.ac.uk","leligou@teihal.gr","laurence.millon@univ-fcomte.fr","simon.podnar@kclj.si","r.j.wakefield@leeds.ac.uk","r.p.schroeder@amc.nl","j.m.blazeby@bristol.ac.uk","mcarrillo@alz.org","magnussen@pulmoresearch.de","pascal.demoly@inserm.fr","antohe@nipne.ro","philippe.poncelet@biocytex.fr","bernd.blobel@mrz.uni-magdeburg.de","bobfreeman@virginmedia.com","swifts@musc.edu","blanker@belvederelaan.nl","jbsoriano2@gmail.com","lebot@vanuatu.com.vu","david.walker@klinikum-konstanz.de","c.legoff@chu.ulg.ac.be","emmanuel.favaloro@-health.nsw.gov.au","kangh@who.int","j.h.dekker@umcg.nl","kerry.hanson@ucr.edu","pfranco@ausl.vda.it","christophe.delclaux@egp.aphp.fr","francois.mullier@uclouvain.be","n.h.t.ten.hacken@umcg.nl","carina.torn@med.lu.se","stephan.hann@boku.ac.at","nbailly@hcmr.gr","francisco.javier.burgos@upc.edu","meyzchen@scut.edu.cn","jason.hockley@nibsc.org","jean_pierre_bouchard@yahoo.fr","miroslaw.karbowiak@chem.uni.wroc.pl","j.hartman@umcg.nl","r.montironi@univpm.it","ragnar.hanas@vgregion.se","p.vandermeer@sanquin.nl","nadia.harbeck@med.uni-muenchen.de","paul.abrams@bui.ac.uk","mj.cruz@vhir.org","lvlidong@epri.sgcc.com.cn","martin.eisenacher@ruhr-uni-bochum.de","infusino.ilenia@asst-fbf-sacco.it","liegl@big.tuwien.ac.at","chrismaher@urogynaecology.com.au","sfshariat@gmail.com","lennart.minthon@skane.se","gianni.cazzaniga@asst-monza.it","kucera@ujf.cas.cz","menno.vanzelm@monash.edu","alexander.von.gontard@uks.eu","c.alhan@vumc.nl","marina.aiello@unipr.it","serrano@lacim.ca","marti.casadesus@udg.edu","dana.clark@cawthron.org.nz","brad_drummond@med.unc.edu","joakim.dillner@ki.se","julian.barth@nhs.net","ashleyc.smith@mail.utoronto.ca","f.riepert@bba.de","jill_tate@health.qld.gov.au","alberto.costa@unicusano.it","ph.g.degroot@umcutrecht.nl","semposch@nih.gov","ildiko.horvath@koranyi.hu","agrau@imaff.cfmac.csic.es","stephan.walterspacher@glkn.de","wim.janssens@uzleuven.be","anette-g.ziegler@helmholtz-muenchen.de","a.langerak@erasmusmc.nl","lopezd@ohio.edu","gabriel.reboux@univ-fcomte.fr","tomas.kalina@lfmotol.cuni.cz","rblanco@humv.es","michael.vogeser@med.uni-muenchen.de","benoit.wallaert@chru-lille.fr","francoise.dignat-george@univ-amu.fr","shintaro.okazaki@uam.es","h.m.boezen@umcg.nl","stefano.bonassi@sanraffaele.it","terslev@dadlnet.dk","jjchen@hawaii.edu","icshlee@inje.ac.kr","zhangxiuyuan917@163.com","arturo.chiti@hunimed.eu","bourgier@igr.fr","p.doesburg@crystal-lab.nl","gunnar.ellingsen@uit.no","gyorgy.marko-varga@bme.lth.se","jo.haviland@icr.ac.uk","jp-cristol@chu-montpellier.fr","mclee@jnu.ac.kr","windischw@kliniken-koeln.de","rklein@toh.ca","martin.himly@sbg.ac.at","pierluigi.meroni@unimi.it","irene.kamara@vub.ac.be","jean-michel.dogne@unamur.be","massimo.colombo@unimi.it","yi.yang@bl-light.com","ruediger.hahn@uni-hohenheim.de","bentzen@humonc.wisc.edu","nicolasg@iarc.fr","schliemann@derma-jena.de","emilio_filippucci@yahoo.it","edurne.irisarri@ehu.eus","peter.achenbach@helmholtz-muenchen.de","jan.damoiseaux@mumc.nl","g.john@nnuh.nhs.uk","matthew.griffiths@ntu.ac.uk","pierre_delanaye@yahoo.fr","carsten.denkert@charite.de","nicola.bizzaro@ass3.sanita.fvg.it","ana.alfirevic@liverpool.ac.uk","marcus.drake@nbt.nhs.uk","massimo.pompili@uniroma1.it","v.h.j.vandervelden@erasmusmc.nl","etienne.cavalier@chu.ulg.ac.be","m.calderon@imperial.ac.uk","mariechristine.bene@chu-nantes.fr","benjamin.brown@mq.edu.au","diego.r.lopez@telefonica.com","bfjaboal@usc.es","b.carter1@which.net","k.fischer@umcutrecht.nl","sdlee@amc.seoul.kr","sdyujinming@163.com","i.m.boudewijn@umcg.nl","jonathan.douxfils@unamur.be","jdiazg@uniovi.es","alfredoantonio.chetta@unipr.it","david.grimwade@genetics.kcl.ac.uk","romaric.lacroix@univ-amu.fr","h.watz@pulmoresearch.de","pellegrino.r@ospedale.cuneo.it","dancarr@liverpool.ac.uk","kkenn@schoen-kliniken.de","eva.comperat@psl.aphp.fr","b.g.muller@amc.uva.nl","m.pazzagli@dfc.unifi.it","schmidt@biofaction.com","sgranica@wum.edu.pl","hvesper@cdc.gov","francois.schellenberg@laposte.net","elena.campello83@gmail.com","john.yarnold@icr.ac.uk","fferreira@worldbank.org","tim.weitzel@uni-bamberg.de","b.wallace@mail.cryst.bbk.ac.uk","gabriel.schaer@ksa.ch","jmiguelalves@gmail.com","nandita.desouza@icr.ac.uk","aris.papageorghiou@obs-gyn.ox.ac.uk","chris.ryerson@hli.ubc.ca","k.cashman@ucc.ie","n.maffulli@qmul.ac.uk","a.heijboer@vumc.nl","christer.janson@medsci.uu.se","freislingh@iarc.fr","mlopez@ira.cinvestav.mx","p.harrison.1@bham.ac.uk","kai.jakobs@comsys.rwth-aachen.de","lucilla.parnetti@unipg.it","cmichotte@bipm.org","manfred.dreyer@asklepios.com","taylorcl3@od.nih.gov","pierantonio.laveneziana@aphp.fr","jinylee@skku.edu","bond-r@email.ulster.ac.uk","manfred.herold@i-med.ac.at","dmagis@chu.ulg.ac.be","giuseppe.lippi@univr.it","samuli.helama@luke.fi","rico.hiemann@b-tu.de","vbianchi@ospedale.al.it","bretagne@univ-paris12.fr","wolfgang.kern@mll.com","lampasona.vito@hsr.it","p.zanen@umcutrecht.nl","g.hanna@imperial.ac.uk","jan.deprest@uzleuven.be","srobertbarros@gmail.com","thomas.beyer@meduniwien.ac.at","h.j.vandervelden@erasmusmc.nl","sebastiaan.engelborghs@uantwerp.be","g.gerrard@imperial.ac.uk","job.harenberg@medma.uni-heidelberg","n.hopkinson@ic.ac.uk","gscano@unifi.it","gunda.koellensperger@univie.ac.at","vittorio.pengo@unipd.it","ludger.klimek@allergiezen-trum.org","clive.taylor@med.usc.edu","bernd.blobel@ehealth-cc.de","hahn@wi-ol.de","andreiarosariobatista@hotmail.com","g.denheeten@lrcb.nl","christel.lamberg-allardt@helsinki.fi","hllee@mail.ncku.edu.tw","c.a.grimbergen@amc.uva.nl","zhangyq@sohu.com","steve.kitchen@sth.nhs.uk","fedebraga83@gmail.com","katrien.devreese@uzgent.be","colesc@doctors.org.uk","claude.lambert@uni-st-etienne.fr","a.angelini@ispo.toscana.it","giovanni.frisoni@unige.ch","xmunoz@vhebron.net","bengt.winblad@ki.se","elaine.gray@nibsc.org","jean-martin.laberge@muhc.mcgill.ca","a.faulkner@sussex.ac.uk","y.liu6@lboro.ac.uk","emma.english@nottingham.ac.uk","lynnr540@aol.com","p.d.white@mds.qmw.ac.uk"], columns=["email"])

In [65]:
main = [e.split(",")[0].lower() for e in df2["main_email"].tolist()]
sent = [e.lower() for e in df_sent["email"]]
new = set([e for e in main if e not in sent and not e==""])
pd.DataFrame(list(new), columns=["email"])

In [72]:
#co_auth = [e.split(",") for e in df2["email"].tolist()]
co_auth = [email.lower() for e_list in df2["email"].tolist() for email in e_list.split(",")]
main_auth = [email.lower() for e_list in df2["main_email"].tolist() for email in e_list.split(",")]
sent_main = sent + main_auth
new_co_auth = set([e for e in co_auth if e not in sent_main and not e==""])
df_new_co_auth = pd.DataFrame(list(new_co_auth), columns=["email"])
print(len(df_new_co_auth))
df_new_co_auth

1972
